In [1]:
import json
from shapely.geometry import Point, Polygon, MultiPolygon
import geojson
import geopandas as gpd
import pandas as pd
import numpy as np
from time import time
import leafmap.foliumap as leafmap
import supervision as sv
from garuda.od import ConfusionMatrix
from IPython.display import Markdown

In [2]:
region = 'mymensingh'
prediction_file = f"../data/predict/processed_labels/{region}/predictions_0.25.geojson"
print(f"Region: {region}")
print(f"Prediction file: {prediction_file}")

Region: mymensingh
Prediction file: ../data/predict/processed_labels/mymensingh/predictions_0.25.geojson


In [3]:
prediction_gdf = gpd.read_file(prediction_file) 
prediction_gdf['yolo_label'] = prediction_gdf['yolo_label'].apply(lambda x: np.array(list(map(float, x.split("_")))))
print(len(prediction_gdf))
print(prediction_gdf.crs)
prediction_gdf.head()



487
EPSG:3857


,x,y,class_name,confidence,yolo_label,geometry
0,10088467,2801567,Zigzag,0.925639,"[2.0, 0.0391915, 0.397093, 0.0543998, 0.397934...","POLYGON ((10087058.215 2801882.208, 10087104.7..."
1,10088467,2801567,Zigzag,0.251513,"[2.0, 0.650263, 0.682439, 0.66422, 0.685413, 0...","POLYGON ((10088926.555 2801009.768, 10088969.2..."
2,10085715,2801567,Zigzag,0.831930,"[2.0, 0.635295, 0.234922, 0.649381, 0.232, 0.6...","POLYGON ((10086129.057 2802378.043, 10086172.1..."
3,10060949,2807071,FCBK,0.833495,"[1.0, 0.387054, 0.794124, 0.396085, 0.807781, ...","POLYGON ((10060604.468 2806171.759, 10060632.0..."
4,10063701,2807071,FCBK,0.454159,"[1.0, 0.36286, 0.18721, 0.369384, 0.175264, 0....","POLYGON ((10063282.228 2808027.387, 10063302.1..."


In [4]:
prediction_gdf.crs.to_string()
print(prediction_gdf.tail(2))

           x        y class_name  confidence  \
485  9997659  2914388     Zigzag    0.534323   
486  9992156  2917140     Zigzag    0.389579   

                                            yolo_label  \
485  [2.0, 0.660672, 0.582588, 0.665293, 0.570155, ...   
486  [2.0, 0.856602, 0.569892, 0.862631, 0.55725, 0...   

                                              geometry  
485  POLYGON ((9998151.19 2914136.114, 9998165.319 ...  
486  POLYGON ((9993246.777 2916926.665, 9993265.21 ...  


In [5]:
prediction_gdf.reset_index(inplace=True, drop=True)
print(prediction_gdf.head(2))

          x        y class_name  confidence  \
0  10088467  2801567     Zigzag    0.925639   
1  10088467  2801567     Zigzag    0.251513   

                                          yolo_label  \
0  [2.0, 0.0391915, 0.397093, 0.0543998, 0.397934...   
1  [2.0, 0.650263, 0.682439, 0.66422, 0.685413, 0...   

                                            geometry  
0  POLYGON ((10087058.215 2801882.208, 10087104.7...  
1  POLYGON ((10088926.555 2801009.768, 10088969.2...  


In [6]:
intersection_gdf = gpd.sjoin(prediction_gdf, prediction_gdf, predicate="intersects")[['index_right']].reset_index()
intersection_gdf = intersection_gdf[intersection_gdf['index'] < intersection_gdf['index_right']]
len(intersection_gdf)

11

In [7]:
def get_iou(row):
    left_polygon = prediction_gdf.iloc[int(row['index'])]['geometry']
    right_polygon = prediction_gdf.iloc[int(row['index_right'])]['geometry']
    left_area = left_polygon.area
    right_area = right_polygon.area
    iou = left_polygon.intersection(right_polygon).area / left_polygon.union(right_polygon).area
    return iou, left_area, right_area

intersection_gdf[['iou', 'left_area', 'right_area']] = intersection_gdf.apply(get_iou, axis=1, result_type="expand")
intersection_gdf = intersection_gdf[intersection_gdf['iou'] > 0.33]
len(intersection_gdf)
print(intersection_gdf.head(2))

     index  index_right       iou    left_area   right_area
14      13           16  0.433693  1867.706764  3723.502401
192    187          189  0.486734  1898.170926  3699.748924


In [8]:
drop_idx = intersection_gdf.apply(lambda x: x['index'] if x['left_area'] < x['right_area'] else x['index_right'], axis=1).astype(int).values
print(len(prediction_gdf))
prediction_gdf.drop(drop_idx, inplace=True)
print(len(prediction_gdf))
display(prediction_gdf.head(2))

487
480


,x,y,class_name,confidence,yolo_label,geometry
0,10088467,2801567,Zigzag,0.925639,"[2.0, 0.0391915, 0.397093, 0.0543998, 0.397934...","POLYGON ((10087058.215 2801882.208, 10087104.7..."
1,10088467,2801567,Zigzag,0.251513,"[2.0, 0.650263, 0.682439, 0.66422, 0.685413, 0...","POLYGON ((10088926.555 2801009.768, 10088969.2..."


In [9]:
final_kilns=prediction_gdf.to_crs(epsg=4326)
display(final_kilns.head(2))

,x,y,class_name,confidence,yolo_label,geometry
0,10088467,2801567,Zigzag,0.925639,"[2.0, 0.0391915, 0.397093, 0.0543998, 0.397934...","POLYGON ((90.61359 24.39719, 90.614 24.39717, ..."
1,10088467,2801567,Zigzag,0.251513,"[2.0, 0.650263, 0.682439, 0.66422, 0.685413, 0...","POLYGON ((90.63037 24.39005, 90.63075 24.38997..."


In [10]:
import os
save_handvalidation_dir = f"../hand_validation/{region}"
os.makedirs(save_handvalidation_dir, exist_ok=True)

In [11]:
len(final_kilns)

480

In [12]:
version = "v1"  
batch_size = 500
for i in range(0, len(final_kilns), batch_size):
    end = min(i+batch_size, len(final_kilns))
    final_kilns.iloc[i:i+batch_size].to_file(f"../hand_validation/{region}/{version}_potential_kiln_{region}_{i}_{end-1}.geojson", driver='GeoJSON')

GARUDA INFO     : Created 480 records
